In [ ]:
!pip install lazypredict
import pandas as pd
import io
from googleapiclient.http import MediaIoBaseDownload
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from lazypredict.Supervised import LazyRegressor
from google.colab import files
pd.reset_option

In [ ]:
df = pd.read_csv('/content/skempi_v2.csv', sep=';')
len(df.columns)

In [ ]:
#удаляем все строки, в которых нет температуры или константы
df = df.dropna(subset=['Temperature', 'Affinity_wt_parsed', 'Affinity_mut_parsed'])
df = df.dropna(axis=1, thresh=len(df) * 0.8) #удаляем столбцы, в которых заполнено менее 80% данных
df

In [ ]:
#кодируем наши данные
for col in ['#Pdb', 'Mutation(s)_PDB', 'Mutation(s)_cleaned',
                    'iMutation_Location(s)', 'Hold_out_proteins',
                    'Protein 1', 'Protein 2', 'Method']:
    freq = df[col].value_counts(normalize=True)
    df[col] = df[col].map(freq)
df

In [ ]:
#вводим бинарные метки
ddG=[]
for index, row in df.iterrows():
    #не стала домножать на температуру и R, так как на знак они не влияет
    G=-math.log(row['Affinity_mut_parsed']/row['Affinity_wt_parsed'])
    if G > 0:
        ddG.append(1)
    else:
        ddG.append(0)
df['ddG']=ddG
df = df.drop(columns=[0, 1], errors='ignore')
df

In [ ]:
df.dtypes

In [ ]:
#удалим всё, что не является числами, температуру переведем в числовой формат
Temperature=[]
Reference=[]
df= df.drop(columns=['Affinity_mut (M)','Affinity_wt (M)', 'Reference'], errors='ignore')
for i in df['Temperature']:
    Temperature.append(float(i.split('(')[0]))
df['Temperature']=Temperature
df.dtypes

In [ ]:
#опционально, так как признаков и так осталось достаточно мало
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import shap

X = df.drop(columns=['ddG'])  # Признаки
y = df['ddG']                # Бинарная метка

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Random Forest для оценки важности признаков
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X, y)
importance = pd.DataFrame({'Feature': X.columns, 'Importance': model_rf.feature_importances_})
print(importance.sort_values('Importance', ascending=False))

# SHAP-анализ для интерпретируемости
explainer = shap.TreeExplainer(model_rf)
shap_values = explainer.shap_values(X)

In [ ]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
X = df.drop(columns=['ddG'])  # Признаки
y = df['ddG']                # Целевая переменная

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Автоматическое обучение и сравнение моделей
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Вывод результатов (сортировка по 'Accuracy')
print(models.sort_values('Accuracy', ascending=False))

In [ ]:
#Посмотрим на обучпемость наших трех лидеров. Начнем с BaggingClassifer
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
model = BaggingClassifier(n_estimators=100, random_state=42, max_samples=200)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', annot_kws={'fontsize': 14})
plt.ylabel('Истинные значения')
plt.xlabel('Предсказанные значения')
plt.title('Матрица ошибок', pad=15)
plt.show()


In [ ]:
report = classification_report(y_test, model.predict(X_test))
print(report)

In [ ]:
#теперь XGBCClassifier
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(X_train.values, y_train.values)
y_pred_model = model.predict(X_test.values)
cm = confusion_matrix(y_test, y_pred_model)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', annot_kws={'fontsize': 14})
plt.ylabel('Истинные значения')
plt.xlabel('Предсказанные значения')
plt.title('Матрица ошибок', pad=15)
plt.show()


In [ ]:
report = classification_report(y_test, model.predict(X_test.values))
print(report)

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
model = lgb.LGBMClassifier()
model.fit(X_train.values, y_train.values)
y_pred_model = model.predict(X_test.values)
cm = confusion_matrix(y_test, y_pred_model)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', annot_kws={'fontsize': 14})
plt.ylabel('Истинные значения')
plt.xlabel('Предсказанные значения')
plt.title('Матрица ошибок', pad=15)
plt.show()

In [ ]:
report = classification_report(y_test, model.predict(X_test.values))
print(report)